In [15]:
import torch
from sam.build_sam import sam_model_registry
from sam.automatic_mask_generator import SamAutomaticMaskGenerator
from sam.predictor import SamPredictor
from sam.build_sam import SAM_Model
from data_builder.build_dataset import PlanetscopeDataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.optimize import linear_sum_assignment
from torch.utils.data import DataLoader

In [4]:
ROOT_PATH = 'C://Users/anind/Dropbox (ASU)/ASU/Kerner-Lab/SAT-SAM(Dataset)/ps_rwanda/all_dataset/'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [16]:
sam = sam_model_registry['vit_l'](checkpoint='sam/checkpoint/sam_vit_l_0b3195.pth')
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

In [17]:
dataset = PlanetscopeDataset(ROOT_PATH, train=False)

In [ ]:
def build_point_grid(n_per_side: int) -> np.ndarray:
    """Generates a 2D grid of points evenly spaced in [0,1]x[0,1]."""
    offset = 1 / (2 * n_per_side)
    points_one_side = np.linspace(offset, 1 - offset, n_per_side)
    points_x = np.tile(points_one_side[None, :], (n_per_side, 1))
    points_y = np.tile(points_one_side[:, None], (1, n_per_side))
    points = np.stack([points_x, points_y], axis=-1).reshape(-1, 2)
    return points

def get_bbox_point_and_inputlabel_prompts(pixel_masks, image_width, image_height, num_boxes, min_distance, box_percentage):
    bbox_prompts = []
    point_prompts = []
    input_labels_prompts = []
    
    if len(pixel_masks) == 0:
        # TODO: We have to create few bounding boxes here with just blank background
        bbox_prompts = generate_random_bounding_boxes(image_width, image_height, num_boxes, min_distance, box_percentage)
    else:
        for mask in pixel_masks:
            # get bounding box from mask
            y_indices, x_indices = np.where(mask > 0)

            # # This is done if we have masks but the labels are all 0, we have to generate random bounding boxes then
            if (len (x_indices) == 0) or (len(y_indices) == 0):
                bbox_prompts = generate_random_bounding_boxes(image_width, image_height, num_boxes, min_distance, box_percentage)
            else:
                x_min, x_max = np.min(x_indices), np.max(x_indices)
                y_min, y_max = np.min(y_indices), np.max(y_indices)
                # add perturbation to bounding box coordinates
                H, W = mask.shape
                x_min = max(0, x_min - np.random.randint(0, 30))
                x_max = min(W, x_max + np.random.randint(0, 30))
                y_min = max(0, y_min - np.random.randint(0, 30))
                y_max = min(H, y_max + np.random.randint(0, 30))
                bbox = [x_min, y_min, x_max, y_max]
                bbox_prompts.append(bbox)

                # Get grid points within the bounding box
                point_grid_per_crop= build_point_grid(10)
                #This forms a 2D grid points and we need to normalize it to the bbox size
                point_grid_per_crop[:, 0] = point_grid_per_crop[:, 0] * (x_max - x_min) + x_min
                point_grid_per_crop[:, 1] = point_grid_per_crop[:, 1] * (y_max - y_min) + y_min
                #Convert the grid points to a integer
                point_grid_per_crop = np.around(point_grid_per_crop).astype(np.float64)
                points_per_crop = np.array([np.array([point_grid_per_crop])])
                points_per_crop = np.transpose(points_per_crop, axes=(0, 2, 1, 3))
                point_prompts.append(points_per_crop)

                input_labels = np.ones_like(points_per_crop[:, :, :, 0], dtype=np.int64)
                input_labels_prompts.append(input_labels)

    return bbox_prompts, point_prompts, input_labels_prompts

In [30]:
def iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def calculate_iou_matrix(target_masks, predicted_masks):
    num_target_masks = len(target_masks)
    num_predicted_masks = len(predicted_masks)

    print("num_target_masks: ", num_target_masks)
    print("num_predicted_masks: ", num_predicted_masks)

    iou_matrix = np.zeros((num_target_masks, num_predicted_masks))
    for i in range(num_target_masks):
        for j in range(num_predicted_masks):
            iou_matrix[i, j] = iou(target_masks[i], predicted_masks[j])

    return iou_matrix

def calculate_iou(target_masks, predicted_masks):
    iou_matrix = calculate_iou_matrix(target_masks, predicted_masks)
    # Use the Hungarian algorithm to find the best assignment
    row_ind, col_ind = linear_sum_assignment(-iou_matrix)

    print("row_ind: ", row_ind)
    print("col_ind: ", col_ind)

    total_iou = 0.0
    for i, j in zip(row_ind, col_ind):
        total_iou += iou_matrix[i, j]

    average_iou = total_iou / len(row_ind)
    return average_iou

def calculate_precision_recall(matched_ious, num_pred_instances, num_gt_instances, threshold=0.5):
    num_true_positives = len(matched_ious[matched_ious >= threshold])
    num_false_positives = num_pred_instances - num_true_positives
    num_false_negatives = num_gt_instances - num_true_positives
    precision = num_true_positives / (num_true_positives + num_false_positives)
    recall = num_true_positives / (num_true_positives + num_false_negatives)
    
    return precision, recall

In [18]:
sam_image, rpn_image, target, ensemble = dataset[0]

In [26]:
sam_image = np.array(sam_image)
sam_image.shape

(448, 448, 3)

In [32]:
masks = mask_generator.generate(sam_image)

TypeError: list indices must be integers or slices, not str

In [34]:
masks[0]['segmentation']

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True]])

In [35]:
iou_score = calculate_iou(target_masks=np.array(target['masks']), predicted_masks=np.array(masks[0]['segmentation'], dtype=np.uint8))
print("Average IoU Score: ", iou_score)

num_target_masks:  1
num_predicted_masks:  448
row_ind:  [0]
col_ind:  [417]
Average IoU Score:  0.02261836500655308


In [ ]:
iou_scores = []

for i, (sam_image, rpn_image, target, ensemble)  in enumerate(dataset): 
    try:
        prediction = sam_automatic_mask_generator.generate(sam_image)

        iou_score = calculate_iou(target_masks=np.array(target['masks']), predicted_masks=np.array(high_res_masks))
        print("Image Id: ", i, " Average IoU Score: ", iou_score)

        iou_scores.append(iou_score)
    except:
        print("Error in image: ", i)
        continue

In [ ]:
np.mean(iou_scores)